# Sentiment Analysis - Machine Learning and Basic Deep Neural Network Models

We have already discussed that sentiment analysis, also popularly known as opinion analysis or opinion mining is one of the most important applications of NLP. The key idea is to predict the potential sentiment of a body of text based on the textual content. In this sub-unit, we will be exploring supervised learning models. 

![](https://github.com/dipanjanS/nlp_workshop_dhs18/blob/master/Unit%2012%20-%20Project%209%20-%20Sentiment%20Analysis%20-%20Supervised%20Learning/sentiment_cover.png?raw=1)

Another way to build a model to understand the text content and predict the sentiment of the text based reviews is to use supervised machine learning. To be more specific, we will be using classification models for solving this problem. We will be building an automated sentiment text classification system in subsequent sections. The major steps to achieve this are mentioned as follows.

1.	Prepare train and test datasets (optionally a validation dataset)
2.	Pre-process and normalize text documents
3.	Feature Engineering 
4.	Model training
5.	Model prediction and evaluation

These are the major steps for building our system. Optionally the last step would be to deploy the model in your server or on the cloud. The following figure shows a detailed workflow for building a standard text classification system with supervised learning (classification) models.

![](https://github.com/dipanjanS/nlp_workshop_dhs18/blob/master/Unit%2012%20-%20Project%209%20-%20Sentiment%20Analysis%20-%20Supervised%20Learning/sentiment_classifier_workflow.png?raw=1)


In our scenario, documents indicate the movie reviews and classes indicate the review sentiments which can either be positive or negative making it a binary classification problem. We will build models using both traditional machine learning methods and newer deep learning in the subsequent sections. 

In [1]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

modin 0.2.2 has requirement pandas==0.23.4, but you'll have pandas 0.24.2 which is incompatible.
jupyter-console 5.2.0 has requirement prompt_toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.9 which is incompatible.
dash 0.42.0 has requirement Flask>=0.12, but you'll have flask 0.10.1 which is incompatible.
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
modin 0.2.2 has requirement pandas==0.23.4, but you'll have pandas 0.24.2 which is incompatible.
jupyter-console 5.2.0 has requirement prompt_toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.9 which is incompatible.
dash 0.42.0 has requirement Flask>=0.12, but you'll have flask 0.10.1 which is incompatible.
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
modin 0.2.2 has requirement pandas==0.23.4, but you'll have pandas 0.24.

True

# Load and View Dataset

In [2]:
import pandas as pd

dataset = pd.read_csv(r'https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2011%20-%20Sentiment%20Analysis%20-%20Unsupervised%20Learning/movie_reviews.csv.bz2')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null object
dtypes: object(2)
memory usage: 781.3+ KB


In [3]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Build Train and Test Datasets

In [4]:
# build train and test datasets
reviews = dataset['review'].values
sentiments = dataset['sentiment'].values

train_reviews = reviews[:35000]
train_sentiments = sentiments[:35000]

test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]

# Text Wrangling & Normalization

In [5]:
import contractions
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata


def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    norm_docs.append(doc)
  
  return norm_docs

In [6]:
%%time

norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

100%|██████████| 15000/15000 [00:07<00:00, 1994.03it/s]

CPU times: user 25.3 s, sys: 313 ms, total: 25.6 s
Wall time: 25.6 s


# Traditional Supervised Machine Learning Models

## Feature Engineering

In [7]:
%%time

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# build BOW features on train reviews
cv = CountVectorizer(binary=False, min_df=5, max_df=1.0, ngram_range=(1,2))
cv_train_features = cv.fit_transform(norm_train_reviews)


# build TFIDF features on train reviews
tv = TfidfVectorizer(use_idf=True, min_df=5, max_df=1.0, ngram_range=(1,2),
                     sublinear_tf=True)
tv_train_features = tv.fit_transform(norm_train_reviews)

CPU times: user 50.5 s, sys: 1.52 s, total: 52 s
Wall time: 52.3 s


In [8]:
%%time

# transform test reviews into features
cv_test_features = cv.transform(norm_test_reviews)
tv_test_features = tv.transform(norm_test_reviews)

CPU times: user 12.5 s, sys: 167 ms, total: 12.7 s
Wall time: 12.7 s


In [9]:
print('BOW model:> Train features shape:', cv_train_features.shape, ' Test features shape:', cv_test_features.shape)
print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

BOW model:> Train features shape: (35000, 194922)  Test features shape: (15000, 194922)
TFIDF model:> Train features shape: (35000, 194922)  Test features shape: (15000, 194922)


## Model Training, Prediction and Performance Evaluation

### Try out Logistic Regression

The logistic regression model is actually a statistical model developed by statistician
David Cox in 1958. It is also known as the logit or logistic model since it uses the
logistic (popularly also known as sigmoid) mathematical function to estimate the
parameter values. These are the coefficients of all our features such that the overall loss
is minimized when predicting the outcome—

In [10]:
%%time

# Logistic Regression model on BOW features
from sklearn.linear_model import LogisticRegression

# instantiate model
lr = LogisticRegression(penalty='l2', max_iter=500, C=1, solver='lbfgs', random_state=42)

# train model
lr.fit(cv_train_features, train_sentiments)

# predict on test data
lr_bow_predictions = lr.predict(cv_test_features)

CPU times: user 4min 37s, sys: 1min 40s, total: 6min 18s
Wall time: 1min


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [11]:
from sklearn.metrics import confusion_matrix, classification_report

labels = ['negative', 'positive']
print(classification_report(test_sentiments, lr_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, lr_bow_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    negative       0.90      0.90      0.90      7490
    positive       0.90      0.91      0.90      7510

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



,negative,positive
negative,6754,736
positive,711,6799


In [12]:
%%time

# Logistic Regression model on TF-IDF features

# train model
lr.fit(tv_train_features, train_sentiments)

# predict on test data
lr_tfidf_predictions = lr.predict(tv_test_features)

CPU times: user 11.6 s, sys: 4.67 s, total: 16.2 s
Wall time: 2.84 s


In [13]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, lr_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, lr_tfidf_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    negative       0.91      0.89      0.90      7490
    positive       0.90      0.91      0.90      7510

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



,negative,positive
negative,6694,796
positive,665,6845


### Try out Random Forest

Decision trees are a family of supervised machine learning algorithms that can represent
and interpret sets of rules automatically from the underlying data. They use metrics like
information gain and gini-index to build the tree. However, a major drawback of decision
trees is that since they are non-parametric, the more data there is, greater the depth of
the tree. We can end up with really huge and deep trees that are prone to overfitting. The
model might work really well on training data, but instead of learning, it just memorizes
all the training samples and builds very specific rules to them. Hence, it performs really
poorly on the test data. Random forests try to tackle this problem.

A random forest is a meta-estimator or an ensemble model that fits a number of
decision tree classifiers on various sub-samples of the dataset and uses averaging to
improve the predictive accuracy and control over-fitting. The sub-sample size is always
the same as the original input sample size, but the samples are drawn with replacement
(bootstrap samples). In random forests, all the trees are trained in parallel (bagging
model/bootstrap aggregation). Besides this, each tree in the ensemble is built from a
sample drawn with replacement (i.e., a bootstrap sample) from the training set. Also,
when splitting a node during the construction of the tree, the split that is chosen is no
longer the best split among all features. Instead, the split that is picked is the best split
among a random subset of the features. T

In [14]:
%%time

# Random Forest model on BOW features
from sklearn.ensemble import RandomForestClassifier

# instantiate model
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)

# train model
rf.fit(cv_train_features, train_sentiments)

# predict on test data
rf_bow_predictions = rf.predict(cv_test_features)

CPU times: user 3min 33s, sys: 1.18 s, total: 3min 34s
Wall time: 31.8 s


In [15]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, rf_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, rf_bow_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    negative       0.86      0.86      0.86      7490
    positive       0.86      0.86      0.86      7510

    accuracy                           0.86     15000
   macro avg       0.86      0.86      0.86     15000
weighted avg       0.86      0.86      0.86     15000



,negative,positive
negative,6406,1084
positive,1080,6430


In [16]:
%%time

# Random Forest model on TF-IDF features

# train model
rf.fit(tv_train_features, train_sentiments)

# predict on test data
rf_tfidf_predictions = rf.predict(tv_test_features)

CPU times: user 3min 21s, sys: 1.02 s, total: 3min 22s
Wall time: 30.6 s


In [17]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, rf_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, rf_tfidf_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    negative       0.85      0.86      0.85      7490
    positive       0.86      0.84      0.85      7510

    accuracy                           0.85     15000
   macro avg       0.85      0.85      0.85     15000
weighted avg       0.85      0.85      0.85     15000



,negative,positive
negative,6458,1032
positive,1175,6335


# Newer Supervised Deep Learning Models

In [18]:
import gensim
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Dense
from sklearn.preprocessing import LabelEncoder

## Prediction class label encoding

In [19]:
le = LabelEncoder()
# tokenize train reviews & encode train labels
tokenized_train = [nltk.word_tokenize(text)
                       for text in norm_train_reviews]
y_train = le.fit_transform(train_sentiments)
# tokenize test reviews & encode test labels
tokenized_test = [nltk.word_tokenize(text)
                       for text in norm_test_reviews]
y_test = le.fit_transform(test_sentiments)

In [20]:
# print class label encoding map and encoded labels
print('Sentiment class label map:', dict(zip(le.classes_, le.transform(le.classes_))))
print('Sample test label transformation:\n'+'-'*35,
      '\nActual Labels:', test_sentiments[:3], '\nEncoded Labels:', y_test[:3])

Sentiment class label map: {'negative': 0, 'positive': 1}
Sample test label transformation:
----------------------------------- 
Actual Labels: ['negative' 'positive' 'negative'] 
Encoded Labels: [0 1 0]


## Feature Engineering with word embeddings

In [21]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [22]:
%%time
# build word2vec model
w2v_num_features = 300
w2v_model = gensim.models.Word2Vec(tokenized_train, size=w2v_num_features, window=150,
                                   min_count=10, workers=4, iter=5)    

2019-08-07 16:37:45,415 : INFO : collecting all words and their counts
2019-08-07 16:37:45,416 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-08-07 16:37:45,881 : INFO : PROGRESS: at sentence #10000, processed 2294933 words, keeping 82417 word types
2019-08-07 16:37:46,381 : INFO : PROGRESS: at sentence #20000, processed 4591079 words, keeping 124832 word types
2019-08-07 16:37:46,867 : INFO : PROGRESS: at sentence #30000, processed 6884452 words, keeping 159825 word types
2019-08-07 16:37:47,108 : INFO : collected 176258 word types from a corpus of 8035381 raw words and 35000 sentences
2019-08-07 16:37:47,109 : INFO : Loading a fresh vocabulary
2019-08-07 16:37:47,225 : INFO : min_count=10 retains 24661 unique words (13% of original 176258, drops 151597)
2019-08-07 16:37:47,226 : INFO : min_count=10 leaves 7763119 word corpus (96% of original 8035381, drops 272262)
2019-08-07 16:37:47,339 : INFO : deleting the raw counts dictionary of 176258 items
2019

2019-08-07 16:38:47,228 : INFO : EPOCH 2 - PROGRESS: at 57.67% examples, 142596 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:38:48,255 : INFO : EPOCH 2 - PROGRESS: at 60.05% examples, 142321 words/s, in_qsize 6, out_qsize 1
2019-08-07 16:38:49,303 : INFO : EPOCH 2 - PROGRESS: at 62.14% examples, 141118 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:38:50,397 : INFO : EPOCH 2 - PROGRESS: at 63.96% examples, 139237 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:38:51,482 : INFO : EPOCH 2 - PROGRESS: at 65.51% examples, 137035 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:38:52,501 : INFO : EPOCH 2 - PROGRESS: at 67.00% examples, 135104 words/s, in_qsize 8, out_qsize 0
2019-08-07 16:38:53,545 : INFO : EPOCH 2 - PROGRESS: at 68.48% examples, 133163 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:38:54,638 : INFO : EPOCH 2 - PROGRESS: at 69.87% examples, 130923 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:38:55,672 : INFO : EPOCH 2 - PROGRESS: at 71.70% examples, 129962 words/s, in_qsiz

2019-08-07 16:39:56,301 : INFO : EPOCH 4 - PROGRESS: at 3.63% examples, 85670 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:39:57,480 : INFO : EPOCH 4 - PROGRESS: at 5.62% examples, 88627 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:39:58,655 : INFO : EPOCH 4 - PROGRESS: at 7.58% examples, 90273 words/s, in_qsize 6, out_qsize 1
2019-08-07 16:39:59,787 : INFO : EPOCH 4 - PROGRESS: at 9.46% examples, 91855 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:40:00,793 : INFO : EPOCH 4 - PROGRESS: at 10.93% examples, 90608 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:40:01,796 : INFO : EPOCH 4 - PROGRESS: at 12.80% examples, 92391 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:40:02,835 : INFO : EPOCH 4 - PROGRESS: at 15.13% examples, 97301 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:40:03,855 : INFO : EPOCH 4 - PROGRESS: at 17.35% examples, 101325 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:40:04,898 : INFO : EPOCH 4 - PROGRESS: at 20.54% examples, 107662 words/s, in_qsize 7, out_qs

2019-08-07 16:41:10,412 : INFO : EPOCH 5 - PROGRESS: at 29.04% examples, 85902 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:41:11,484 : INFO : EPOCH 5 - PROGRESS: at 30.42% examples, 85504 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:41:12,685 : INFO : EPOCH 5 - PROGRESS: at 31.84% examples, 84565 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:41:13,765 : INFO : EPOCH 5 - PROGRESS: at 33.29% examples, 84244 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:41:14,782 : INFO : EPOCH 5 - PROGRESS: at 34.80% examples, 84170 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:41:15,815 : INFO : EPOCH 5 - PROGRESS: at 36.41% examples, 84314 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:41:16,853 : INFO : EPOCH 5 - PROGRESS: at 37.98% examples, 84452 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:41:17,863 : INFO : EPOCH 5 - PROGRESS: at 39.42% examples, 84415 words/s, in_qsize 7, out_qsize 0
2019-08-07 16:41:19,053 : INFO : EPOCH 5 - PROGRESS: at 40.90% examples, 83807 words/s, in_qsize 7, out_

CPU times: user 16min 15s, sys: 4.93 s, total: 16min 20s
Wall time: 4min 14s


In [23]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [24]:
# generate averaged word vector features from word2vec model
avg_wv_train_features = averaged_word2vec_vectorizer(corpus=tokenized_train, model=w2v_model,
                                                     num_features=w2v_num_features)
avg_wv_test_features = averaged_word2vec_vectorizer(corpus=tokenized_test, model=w2v_model,
                                                    num_features=w2v_num_features)

In [25]:
print('Word2Vec model:> Train features shape:', avg_wv_train_features.shape, ' Test features shape:', avg_wv_test_features.shape)

Word2Vec model:> Train features shape: (35000, 300)  Test features shape: (15000, 300)


## Modeling with deep neural networks 

### Building Deep neural network architecture

In [26]:
def construct_deepnn_architecture(num_input_features):
    dnn_model = Sequential()
    dnn_model.add(Dense(512, input_shape=(num_input_features,)))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(256))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(256))
    dnn_model.add(Activation('relu'))
    dnn_model.add(Dropout(0.2))
    
    dnn_model.add(Dense(1))
    dnn_model.add(Activation('sigmoid'))

    dnn_model.compile(loss='binary_crossentropy', optimizer='adam',                 
                      metrics=['accuracy'])
    return dnn_model

In [27]:
w2v_dnn = construct_deepnn_architecture(num_input_features=w2v_num_features)

Instructions for updating:
Colocations handled automatically by placer.


2019-08-07 16:43:49,921 : WARNING : From /anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2019-08-07 16:43:49,977 : WARNING : From /anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/layers/core.py:143: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Visualize sample deep architecture

In [28]:
w2v_dnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               154112    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
__________

### Model Training, Prediction and Performance Evaluation

In [29]:
batch_size = 100
w2v_dnn.fit(avg_wv_train_features, y_train, epochs=10, batch_size=batch_size, 
            shuffle=True, validation_split=0.1, verbose=1)

Train on 31500 samples, validate on 3500 samples
Instructions for updating:
Use tf.cast instead.


2019-08-07 16:43:52,424 : WARNING : From /anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/10
31500/31500 [==============================] - 2s 70us/sample - loss: 0.3267 - acc: 0.8609 - val_loss: 0.3180 - val_acc: 0.8686
Epoch 2/10
31500/31500 [==============================] - 2s 62us/sample - loss: 0.2973 - acc: 0.8759 - val_loss: 0.2991 - val_acc: 0.8829
Epoch 3/10
31500/31500 [==============================] - 2s 59us/sample - loss: 0.2894 - acc: 0.8804 - val_loss: 0.2916 - val_acc: 0.8794
Epoch 4/10
31500/31500 [==============================] - 2s 61us/sample - loss: 0.2803 - acc: 0.8847 - val_loss: 0.3033 - val_acc: 0.8774
Epoch 5/10
31500/31500 [==============================] - 2s 61us/sample - loss: 0.2725 - acc: 0.8866 - val_loss: 0.3012 - val_acc: 0.8760
Epoch 6/10
31500/31500 [==============================] - 2s 62us/sample - loss: 0.2666 - acc: 0.8888 - val_loss: 0.3072 - val_acc: 0.8834
Epoch 7/10
31500/31500 [==============================] - 2s 66us/sample - loss: 0.2591 - acc: 0.8916 - val_loss: 0.3050 - val_acc: 0.8823
Epoch 8/10
31500/31500 [===

In [30]:
y_pred = w2v_dnn.predict_classes(avg_wv_test_features)
predictions = le.inverse_transform(y_pred) 

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, predictions))
pd.DataFrame(confusion_matrix(test_sentiments, predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    negative       0.87      0.88      0.88      7490
    positive       0.88      0.87      0.88      7510

    accuracy                           0.88     15000
   macro avg       0.88      0.88      0.88     15000
weighted avg       0.88      0.88      0.88     15000



,negative,positive
negative,6628,862
positive,984,6526
